# GARCH LSTM Forecasting

I will be making a model to forecast S&P 500 returns as well as using CBOE Volatility Index.

I will get both of these data from yahoo finance.

### Importing libraries and packages

In [50]:
import numpy as np
import pandas as pd

import yfinance
from sklearn.preprocessing import StandardScaler

import torch
from torch import optim
from torch import nn

from arch import arch_model

### Datasets

In [59]:
start_date = "2005-01-01"
end_date = "2025-01-01"

vix = yfinance.download('^VIX', start=start_date, end=end_date, interval='1d', auto_adjust=False)
vix.columns = ['Adj_Close', 'Close', 'High', 'Low', 'Open', 'Volume']
vix.index = pd.to_datetime(vix.index)
vix.dropna(inplace=True)


sp500 = yfinance.download('^GSPC', start=start_date, end=end_date, interval='1d', auto_adjust=False)
sp500.columns = ['Adj_Close', 'Close', 'High', 'Low', 'Open', 'Volume']
sp500.index = pd.to_datetime(sp500.index)

# Preprocess returns: log returns -> detrending -> standardize
sp500['Log_Returns'] = np.log(sp500['Adj_Close']).diff()
sp500['Log_Returns'] = sp500['Log_Returns'] - sp500['Log_Returns'].mean()  # Detrend
returns_scaler = StandardScaler()
sp500['Log_Returns'] = returns_scaler.fit_transform(sp500[['Log_Returns']].fillna(0))

sp500['Volatility'] = sp500['High']/sp500['Low'] - 1
sp500.dropna(inplace=True)


fed_rate = pd.read_csv("EFFR.csv")
fed_rate.columns = ['observation_date', 'Fed_Rate']
fed_rate['observation_date'] = pd.to_datetime(fed_rate['observation_date'])
fed_rate = fed_rate.set_index('observation_date')
fed_rate.dropna(inplace=True)


cpi = pd.read_csv("CPIAUCSL.csv")
cpi.columns = ['observation_date', 'CPI_Rate']
cpi['observation_date'] = pd.to_datetime(cpi['observation_date'])
cpi = cpi.set_index('observation_date')
cpi.dropna(inplace=True)

cpi.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



,CPI_Rate
observation_date,
2005-01-01,191.6
2005-02-01,192.4
2005-03-01,193.1
2005-04-01,193.7
2005-05-01,193.6


### Model initialization

## Hybrid GARCH-LSTM Model
This model combines GARCH (baseline volatility) and LSTM (nonlinear corrections using GARCH residuals and VIX).

In [ ]:
# to be implemented, somehow, (I'll try T_T)